# MST

The pseudocode for the basic algorithm to build a MST of a weighted undirected graph $G$ is below:

```text
Initialize:
  T as a copy of G with all its vertices and none of its edges.
  Count and label components in T

while T has more than one component:
  Combine two components with safe edge.
  Count and label components in T.

T is the Minimum Spanning Tree of G.
```

The algorithm is quite simple as long as we fully grasp the notion of a *safe edge*.

Consider two vertices $u,v\in T$ that are in different components:

$$ \textsf{comp}[u] \neq\textsf{comp}[v] $$

Because $u,v$ are in different components in $T$, there is no edge between them. By definition $T[u][v] = T[v][u] = \infty$. Vertex $u$ maybe connected to other vertices in $T$ (excluding $v$ of course). Likewise, $v$ may be connect to other vertices in $T$ (excluding $u$). Let's write:

\begin{align}
\mathscr{U} & = \text{set of all vertices in }T\text{ reachable from } u \\
\mathscr{V} & = \text{set of all vertices in }T\text{ reachable from } v
\end{align}

Since $\mathscr{U}$ and $\mathscr{V}$ are two different components, they have absolutely no vertex in common, i.e., $\mathscr{U}\cap\mathscr{V} = \varnothing$. These two sets are defined over vertices of graph $T$; however $T$ has the same vertices as the input graph $G$, and so for any vertex $w$:

$$ \text{if}\ w\in \mathscr{U}\cup\mathscr{V}\ \Rightarrow\ w\in G  $$

Recalling that $G=(V,E)$, i.e., the graph is a pair two sets: vertices $V$ and edges $E$, let $\mathscr{E} \subseteq E$ such that

$$\mathscr{E} = \{ (a,b) \in E\ |\ a\in\mathscr{U}\ \text{and}\  b\in \mathscr{V} \}
$$

This is the set of edges in $G$ whose vertices are in different components in $T$.

The safe edge between $\textsf{comp}[u]$ and $\textsf{comp}[v]$ is the edge in $\mathscr{E}$ with the least weight.


## All we need is $\mathscr{U}$ and $\mathscr{V}$

Let's say that somehow we obtain sets $\mathscr{U}$ and $\mathscr{V}$, i.e., the vertices of two different components in $T$. How can we find the safe edge between some vertex of $\mathscr{U}$ and some vertex of  $\mathscr{V}$ that will combine these two components?

The answer is to look at the input graph $G$:

\begin{align}
\forall a \in \ & \mathscr{U}: \\
\forall  b & \in \mathscr{V}:  \\
\text{ if } & G[a][b] < \infty: \\
& G[a][b]\text{ candidate safe edge}
\end{align}

After we produce $\mathscr{U}$ and $\mathscr{V}$ we look at edges between their vertices in the input graph $G$. This is a simple matter of searching for a minimum value. Let's assume that `componentU` and `componentV` are arrays corresponding to $\mathscr{U}$ and $\mathscr{V}$. It takes a few lines of code to find the smallest edge between these two components:

```python
smallest_weight = float('inf')
smallest_edge = []
for a in componentU:
  for b in componentV:
    if G[a][b] < float('inf'):
      if G[a][b] < smallest_weight:
        smallest_weight = G[a][b]
        smallest_edge = [a,b]
```

At the and of the loops above we have identified the safe edge between $\mathscr{U}$ and $\mathscr{V}$. The corresponding vertices are `smallest_edge[0]` and `smallest_edge[1]`. The weight of the edge is, of course, `smallest_edge` but can also be found in the adjacency matrix at

```python
G[smallest_edge[0]][ smallest_edge[1]]
```

All we need to do is to add this edge to $T$


```python
T[smallest_edge[0]][ smallest_edge[1]] = G[smallest_edge[0]][ smallest_edge[1]] # and
T[smallest_edge[1]][ smallest_edge[0]] = G[smallest_edge[1]][ smallest_edge[0]]
```
then re-count and re-label the components in T, and repeat until we are down to one component.

## Assignment

Implement the algorithm described here, to find the minimum spanning tree of a weighted undirected graph $G$, given as an *adjacency matrix.*

In [ ]:
# Helper methods for MST
def reachability_iterative(s, G, visited):
    """Finds all vertices reachable from a specific vertex in an undirected graph.
    This method works iteratively using a stack, only visiting unmarked vertices.

    Inputs
    ------
    s : int
        Vertex label to explore reachability
    G : list
        Adjacency matrix for the graph
    visited : list
        List of marked vertices (boolean values or True/False)

    The method modifies the `visited` list to track visited vertices.
    """
    stack = [s]              # Initialize the stack
    visited[s] = True        # Mark the starting vertex
    while stack:             # Continue as long as the stack is not empty
        u = stack.pop()      # Pop the last item from the stack
        for v in range(len(G[u])):   # Loop over neighbors of u
            # If there's an edge and v is unmarked
            if G[u][v] < float('inf') and not visited[v]:
                visited[v] = True    # Mark v as visited
                stack.append(v)      # Add v to the stack


def count_and_label_components(G):
    """Counts and labels components in an undirected graph represented as an adjacency matrix.

    Inputs
    ------
    G : list
        Adjacency matrix for the graph

    Returns
    -------
    component_id : int
        Number of components
    components : list
        List of component labels for each vertex
    """
    n = len(G)                       # Number of vertices in the graph
    components = [-1] * n            # Component labels for each vertex, initialized to -1
    visited = [False] * n            # Tracking which vertices have been visited
    component_id = 0                 # Component counter

    for v in range(n):               # For every vertex in the graph
        if not visited[v]:            # If the vertex hasn't been visited
            reachability_iterative(v, G, visited)  # Find all vertices in this component
            for i in range(n):       # Label all vertices reachable from v with the current component id
                if visited[i] and components[i] == -1:
                    components[i] = component_id
            component_id += 1        # Increment component counter for next component

    return component_id, components  # Return the number of components and component labels

In [ ]:
def mst(G):
    """
    G is a graph represented as an adjacency matrix
    """
    # Get num of vertices
    numVertices = len(G)
    # Make a copy of the graph with no edges
    T = [[float('inf') for _ in range(numVertices)] for _ in range(numVertices)]

    # Get the count for the number of components and the list of each components
    count, components = count_and_label_components(T)

    # Loop until one because one means the graph is fully connected
    while count > 1:

        # Initialize comps to hold the list of lists for each components vertices
        # At position i is the ith components list of vertices that exist
        comps = [[] for _ in range(count)]

        # Add the vertices for each part of the comps list
        for v in range(numVertices):
            comps[components[v]].append(v)

        # Initialize smallest weight and the smallest edge to find
        # the cheapest safe edge
        smallest_weight = float('inf')
        smallest_edge = []

        # Loop over the first and second component
        for comp_one in range(0, count):
            for comp_two in range(comp_one + 1, count):

                # Loop over all the options for the components
                for a in comps[comp_one]:
                    for b in comps[comp_two]:
                        # If there is a valid edge
                        if G[a][b] < float('inf'):
                            # And its "cheaper" than the current smallest
                            if G[a][b] < smallest_weight:
                                # Update the smallest
                                smallest_weight = G[a][b]
                                smallest_edge = [a,b]

        # add the smallest edge we've found to the new MST.
        u, v = smallest_edge[0], smallest_edge[1]
        T[u][v] = G[u][v]
        T[v][u] = G[v][u]

        # recount
        count, components = count_and_label_components(T)

    return T


In [ ]:
# Sample use case

G1 = [[float('inf'), 2, 4, float('inf'), 1, 7, 3],
    [2, float('inf'), 5, 6, float('inf'), 3, 8],
    [4, 5, float('inf'), 9, 3, float('inf'), 2],
    [float('inf'), 6, 9, float('inf'), 7, 4, float('inf')],
    [1, float('inf'), 3, 7, float('inf'), 2, 6],
    [7, 3, float('inf'), 4, 2, float('inf'), 5],
    [3, 8, 2, float('inf'), 6, 5, float('inf')]]

T1 = mst(G1)

for row in T1:
    print(row)
print()


G2 = [[float('inf'), 2, 4, float('inf')],
    [2, float('inf'), 3, 6],
    [4, 3, float('inf'), 5],
    [float('inf'), 6, 5, float('inf')]]

T2 = mst(G2)

for row in T2:
    print(row)

[inf, 2, inf, inf, 1, inf, 3]
[2, inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, inf, 2]
[inf, inf, inf, inf, inf, 4, inf]
[1, inf, inf, inf, inf, 2, inf]
[inf, inf, inf, 4, 2, inf, inf]
[3, inf, 2, inf, inf, inf, inf]

[inf, 2, inf, inf]
[2, inf, 3, inf]
[inf, 3, inf, 5]
[inf, inf, 5, inf]


# Reading material

[Chapter 7](https://jeffe.cs.illinois.edu/teaching/algorithms/book/07-mst.pdf) of the Jeff Erickson's *Algorithms*.